<a href="https://colab.research.google.com/github/Astridgiraldo/proyecto_Rag/blob/main/Modelo_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Descargas**

In [ ]:
%%capture
!pip install langchain pypdf openai chromadb tiktoken

In [ ]:
%%capture
!pip install -U langchain-community

In [ ]:
%%capture
!pip install PyPDF2

In [ ]:
%%capture
!pip install openai

In [ ]:
%%capture
!pip install --upgrade openai

In [ ]:
%%capture
!pip install -U langchain-openai

# **Documentos**

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from google.colab import drive

#  Directorio donde están almacenados tus PDFs en Google Drive
pdf_directory = '/content/drive/MyDrive/prueba y documentos '  # Cambia esto a la ruta correcta

# Obtener la lista de archivos PDF en el directorio
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

ml_papers = []

# Cargar cada archivo PDF
for i, pdf_file in enumerate(pdf_files):
    file_path = os.path.join(pdf_directory, pdf_file)

    # Cargar el PDF con PyPDFLoader
    loader = PyPDFLoader(file_path)
    documents = loader.load()


    ml_papers.extend(documents)
    print(f'Cargado {pdf_file}')

# Ver el número de documentos cargados
print(f'Se han cargado {len(ml_papers)} paginas.')

Cargado Documento de Anexos_1.pdf
Cargado maiz_y_frijol_biofortificados_intercalados_con_cafe.pdf
Cargado articulo 3 .pdf
Se han cargado 51 paginas.


In [ ]:
print(len(ml_papers))

51


In [ ]:
type(ml_papers), len(ml_papers), ml_papers[1]

(list,
 51,
 Document(metadata={'source': '/content/drive/MyDrive/prueba y documentos /Documento de Anexos_1.pdf', 'page': 1, 'page_label': '2'}, page_content='TABLA DE CONTENIDO \nAnexo No. 1: Cronograma del Proyecto 1 \nAnexo No. 2: Estructura de Desglose del Trabajo (EDT) 2 \nAnexo No. 3: Cuestionario de Levantamiento de Información para Cenicafe 4 \nAnexo No. 4: Cuestionario de Levantamiento de Información para Caficultores y \nGrandes Empresas 5 \nAnexo No. 5: Análisis del Modelo BPSC (Análisis del contexto actual y problemática) 8 \nAnexo No. 6: Análisis del Modelo BPSC (Análisis de la solución desde la Ingeniería) 10 \nAnexo No. 7: Diagrama de Arquitectura 13 \nAnexo No. 8: Requerimientos Funcionales y no Funcionales 14 \nAnexo No. 9: Diagrama BPMN 15 \nAnexo No. 10: Actas de reunión 16 \nAnexo No. 11: Casos de usos 16 \nAnexo No. 12: Modelo Entidad Relación 17 \nAnexo No. 13 :Github 18 \nAnexo No. 14: Prototipos 18 \nAnexo No. 15: Documento de Arquitectura 18 '))

# **Chucks**

In [ ]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1550, ## cada texto va a tener un tamaño de 1500 caracteres
    chunk_overlap=200,
    length_function=len
    )

documents = text_splitter.split_documents(ml_papers)

In [ ]:
len(documents),documents[5]

(87,
 Document(metadata={'source': '/content/drive/MyDrive/prueba y documentos /Documento de Anexos_1.pdf', 'page': 4, 'page_label': '5'}, page_content='- Diseñar estructura de nodos y transacciones\n3.2. Diseño del sistema de NFT \n- Definir características de los NFTs\n- Establecer protocolo de emisión y transferencia\n3.3. Integración de Blockchain y NFT para Trazabilidad \n- Diseñar flujos de datos entre sistemas\n- Establecer puntos de control y autenticidad\n3.4. Diseño de Interfaz de Usuario y UX \n- Crear prototipos de interfaces\n- Definir navegabilidad y experiencia del usuario\n4. Documentación\n4.1. Elaboración de Documentación Técnica\n- Documentar estructuras y patrones de diseño\n- Detallar procesos de implementación y configuración\n4.2. Guías y Manuales para Usuarios \n- Crear guías paso a paso para usuarios finales\n- Desarrollar documentación de resolución de problemas\n4.3. Diagramas y Especificaciones de la Arquitectura \n- Realizar diagramas de flujo y estructura\

In [ ]:
import openai
print(dir(openai))

['APIConnectionError', 'APIError', 'APIResponse', 'APIResponseValidationError', 'APIStatusError', 'APITimeoutError', 'AssistantEventHandler', 'AsyncAPIResponse', 'AsyncAssistantEventHandler', 'AsyncAzureOpenAI', 'AsyncClient', 'AsyncOpenAI', 'AsyncStream', 'Audio', 'AuthenticationError', 'AzureOpenAI', 'BadRequestError', 'BaseModel', 'ChatCompletion', 'Client', 'Completion', 'ConflictError', 'ContentFilterFinishReasonError', 'Customer', 'DEFAULT_CONNECTION_LIMITS', 'DEFAULT_MAX_RETRIES', 'DEFAULT_TIMEOUT', 'DefaultAsyncHttpxClient', 'DefaultHttpxClient', 'Deployment', 'Edit', 'Embedding', 'Engine', 'ErrorObject', 'File', 'FineTune', 'FineTuningJob', 'HttpxBinaryResponseContent', 'Image', 'InternalServerError', 'LengthFinishReasonError', 'Model', 'Moderation', 'NOT_GIVEN', 'NoneType', 'NotFoundError', 'NotGiven', 'Omit', 'OpenAI', 'OpenAIError', 'PermissionDeniedError', 'ProxiesTypes', 'RateLimitError', 'RequestOptions', 'Stream', 'Timeout', 'Transport', 'UnprocessableEntityError', 'VER

# **Embeddings y base de datos vectorial.**

In [ ]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass('Enter the secret value: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

Enter the secret value: ··········


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002')  ## Embeddings

## creacion de base de datos vectorial usando chroma
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

### Configuracion del recuperador de documentos
retriever = vectorstore.as_retriever(
    search_kwargs={"k":3}

    )

<ipython-input-15-7829bdf83726>:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002')  ## Embeddings


# **Modelo de chat - Consulta de informacion**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

chat = ChatOpenAI(
    openai_api_key = OPENAI_API_KEY,
    model_name= 'gpt-3.5-turbo' , ## modelo
    temperature= 0.0 ## control de aleatoriedad de las respuestas, es decir no queremos que el modelo sea creativo

)
## combinacion del modelo generativo con el de recuperacion
qa_chain = RetrievalQA.from_chain_type(
    llm= chat,
    chain_type= "stuff", ## se especifica la manera como se debe integrar la informacion recuperada tambien esta map-reduce y refine
    retriever=retriever
)



In [ ]:
query = " segun el avance tecnico cenicafe 547 que se  realizo para determinar los niveles de acumulacion de zinc de hibrido biofortificado  ?"
qa_chain.run(query)

<ipython-input-16-ea2e85a2a5f7>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run(query)


'Se recolectaron aleatoriamente muestras de granos de maíz con 14% de humedad en cuatro estaciones experimentales de Cenicafé para determinar los niveles de acumulación de zinc del híbrido de maíz blanco biofortificado (SGBIOH2).'

In [ ]:
query = " segun el avance tecnico cenicafe 547 cuales son algunas conclusiones obtenidas  ?"
qa_chain.run(query)

'Según el avance técnico Cenicafé 547, algunas conclusiones obtenidas son que se ha realizado un esfuerzo mundial para aumentar la calidad nutricional de los principales cultivos alimentarios a través de la biofortificación, desarrollando semillas mejoradas con altos contenidos de micronutrientes esenciales. Además, se destaca la importancia de la trazabilidad del café y las prácticas de certificación en la industria cafetera, así como la necesidad de fortalecer el sector agropecuario para mitigar el desplazamiento de los jóvenes rurales hacia áreas urbanas.'

In [ ]:
query = " seguun  la propuesta de prototipo de software para la trazabilidad de los procesos de produccion  de cafe a traves de tecnologias BLOCKCHAIN, como puedo implementar esta tecnologuia  .  ?"
qa_chain.run(query)

'Según la propuesta de prototipo de software para la trazabilidad de los procesos de producción de café a través de tecnologías blockchain, para implementar esta tecnología en tu empresa o proyecto, necesitarías seguir los siguientes pasos:\n\n1. **Investigación y comprensión de blockchain:** Antes de implementar la tecnología, es importante que te familiarices con cómo funciona blockchain y cómo puede aplicarse a la trazabilidad de la producción de café.\n\n2. **Identificación de los procesos a trazar:** Define claramente cuáles son los procesos de producción de café que deseas trazar y qué información relevante deseas almacenar en la cadena de bloques.\n\n3. **Selección de la plataforma blockchain:** Elige la plataforma blockchain que mejor se adapte a tus necesidades. Puedes considerar Ethereum, Hyperledger, entre otras.\n\n4. **Desarrollo del software:** Desarrolla el software que permitirá la trazabilidad de los procesos de producción de café utilizando la tecnología blockchain. P

In [ ]:
query = "segun propuesta de prototipo desoftware para la trazabilidad de los procesos de produccion  de cafe a traves de tecnologias BLOCKCHAIN,  Que es BPMN y para que sirve y como puedo implementarlo"
qa_chain.run(query)

'Según la propuesta de prototipo de software para la trazabilidad de los procesos de producción de café a través de tecnologías Blockchain, BPMN (Business Process Model and Notation) es un estándar de modelado utilizado para describir cómo las actividades de un proceso se relacionan entre sí. \n\nBPMN es una herramienta que se utiliza para modelar gráficamente los procesos de negocios, permitiendo una visualización clara y comprensible de cómo se desarrollan las actividades en un proceso. \n\nPara implementar BPMN en el desarrollo de software, puedes utilizar herramientas de modelado BPMN como Bizagi, Lucidchart, o incluso software especializado en BPMN como Camunda. Estas herramientas te permitirán crear diagramas BPMN de tus procesos de producción de café de manera visual y detallada, lo que facilitará la comprensión y el diseño de tu sistema de trazabilidad.'

In [ ]:
query = "segun el avance tecnico Semillas biofortificadas que son Las semillasbiofortificadas de maíz y fríjol?"
qa_chain.run(query)

<ipython-input-16-157be9be85ad>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run(query)


'Las semillas biofortificadas de maíz y fríjol son variedades mejoradas genéticamente para tener un mayor contenido de nutrientes como zinc y hierro. Estas semillas han sido seleccionadas para garantizar un alto potencial productivo y nutricional en las condiciones específicas de la zona cafetera colombiana.'

In [ ]:
query = (" cuales son Las principales tendencias relativas a la producción de cafés sostenibles y la vinculaciónde las nuevas"
    "generaciones en la producción de café sostenible para el fortalecimiento Del empalme generacional en el municipio de Aguadas (Caldas)?")
qa_chain.run(query)

'Las principales tendencias relativas a la producción de cafés sostenibles y la vinculación de las nuevas generaciones en la producción de café sostenible para el fortalecimiento del empalme generacional en el municipio de Aguadas (Caldas) incluyen la necesidad de implementar estrategias que fomenten la sostenibilidad ambiental, social y económica en la producción de café. Además, es importante promover la certificación de prácticas sostenibles, el uso de tecnologías amigables con el medio ambiente y la creación de incentivos para atraer a las nuevas generaciones a participar en la caficultura de manera sostenible. Sin embargo, es crucial abordar la falta de interés de los jóvenes en continuar con la tradición cafetera familiar y encontrar formas innovadoras de involucrarlos en el sector.'

In [ ]:
query = "segun el avance tecnico cenicafe 547 cuanto y donde se encontro el nivel de mas alto de zinc en las muestras recolectadas que contenian 14% de humedas?"
qa_chain.run(query)

<ipython-input-18-93f339b0dcde>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run(query)


'Según el avance técnico de Cenicafe 547, el nivel más alto de zinc en las muestras recolectadas que contenían 14% de humedad se encontró en la localidad de El Rosario, con un contenido promedio de zinc de 40,18 mg kg-1.'

In [ ]:
query = "en que porcentaje y cuales son los oficiones que los estudiantes de las institiciones rurales del municipio de aguadas caldas proyectan como proyecto de vida?"
qa_chain.run(query)

<ipython-input-17-493e89179777>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run(query)


'El 60% de los egresados de Instituciones Educativas rurales del municipio de Aguadas, Caldas, se proyectan a emprender en la venta de servicios como barberos, estilistas, mecánicos, y comerciantes. Mientras que el 25% de ellos visualizan sus proyectos de vida en oficios y servicios diferentes a la producción y comercialización de café.'

In [ ]:
query = " principales caracteristicas del azuar?"
qa_chain.run(query)